# PySpark Linear Regression

Based on **Krish Naik** video: https://www.youtube.com/watch?v=_C8kWso4ne4

## DataBricks version (no internet notes)

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Practise').getOrCreate()
spark.sparkContext.setLogLevel("WARN")

In [4]:
# File location and type
file_location = "data/tips.csv" # Databricks file_location "/FileStore/tables//test1.csv"
file_type = "csv"

# Theapplied options are for CSV files. For other file types, these will be ignored.
df = spark.read.csv(file_location, header=True, inferSchema=True)

In [5]:
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [6]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [7]:
# Handling Categorical Features

from pyspark.ml.feature import StringIndexer

In [8]:
indexer = StringIndexer(inputCols=["sex", "smoker", "day", "time"], outputCols=["sex_indexed", "smoker_indexed", "day_indexed", "time_indexed"])
df_r = indexer.fit(df).transform(df)

In [9]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=[ 
                'tip',
                'size',
                'sex_indexed',
                'smoker_indexed',
                'day_indexed',
                'time_indexed'],
                outputCol = "Independent Features")

output = featureassembler.transform(df_r)

In [10]:
output.select('Independent Features').show()

+--------------------+
|Independent Features|
+--------------------+
|[1.01,2.0,1.0,0.0...|
|[1.66,3.0,0.0,0.0...|
|[3.5,3.0,0.0,0.0,...|
|[3.31,2.0,0.0,0.0...|
|[3.61,4.0,1.0,0.0...|
|[4.71,4.0,0.0,0.0...|
|[2.0,2.0,0.0,0.0,...|
|[3.12,4.0,0.0,0.0...|
|[1.96,2.0,0.0,0.0...|
|[3.23,2.0,0.0,0.0...|
|[1.71,2.0,0.0,0.0...|
|[5.0,4.0,1.0,0.0,...|
|[1.57,2.0,0.0,0.0...|
|[3.0,4.0,0.0,0.0,...|
|[3.02,2.0,1.0,0.0...|
|[3.92,2.0,0.0,0.0...|
|[1.67,3.0,1.0,0.0...|
|[3.71,3.0,0.0,0.0...|
|[3.5,3.0,1.0,0.0,...|
|(6,[0,1],[3.35,3.0])|
+--------------------+
only showing top 20 rows



In [11]:
finalized_data = output.select("Independent Features", "total_bill")

In [12]:
finalized_data.show()

+--------------------+----------+
|Independent Features|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|[1.66,3.0,0.0,0.0...|     10.34|
|[3.5,3.0,0.0,0.0,...|     21.01|
|[3.31,2.0,0.0,0.0...|     23.68|
|[3.61,4.0,1.0,0.0...|     24.59|
|[4.71,4.0,0.0,0.0...|     25.29|
|[2.0,2.0,0.0,0.0,...|      8.77|
|[3.12,4.0,0.0,0.0...|     26.88|
|[1.96,2.0,0.0,0.0...|     15.04|
|[3.23,2.0,0.0,0.0...|     14.78|
|[1.71,2.0,0.0,0.0...|     10.27|
|[5.0,4.0,1.0,0.0,...|     35.26|
|[1.57,2.0,0.0,0.0...|     15.42|
|[3.0,4.0,0.0,0.0,...|     18.43|
|[3.02,2.0,1.0,0.0...|     14.83|
|[3.92,2.0,0.0,0.0...|     21.58|
|[1.67,3.0,1.0,0.0...|     10.33|
|[3.71,3.0,0.0,0.0...|     16.29|
|[3.5,3.0,1.0,0.0,...|     16.97|
|(6,[0,1],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [13]:
from pyspark.ml.regression import LinearRegression
# train test split
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol = "Independent Features", labelCol="total_bill")
regressor = regressor.fit(train_data)

23/06/28 07:31:48 WARN Instrumentation: [9fe546ef] regParam is zero, which might cause numerical instability and overfitting.
23/06/28 07:31:48 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/06/28 07:31:48 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [14]:
regressor.coefficients

DenseVector([3.2918, 3.5327, -0.7218, 2.284, -0.1233, -2.1187])

In [15]:
regressor.intercept

1.070394653656759

In [16]:
# Predictions
pred_results = regressor.evaluate(test_data)

In [17]:
# Final comparison
pred_results.predictions.show()

+--------------------+----------+------------------+
|Independent Features|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,1],[1.25,2.0])|     10.07| 12.25051403618172|
|(6,[0,1],[1.75,2.0])|     17.82|13.896437645869074|
| (6,[0,1],[2.0,2.0])|     12.69| 14.71939945071275|
| (6,[0,1],[2.0,2.0])|     13.37| 14.71939945071275|
| (6,[0,1],[2.0,3.0])|     16.31| 18.25205462986604|
|(6,[0,1],[2.24,3.0])|     16.04| 19.04209796251597|
| (6,[0,1],[2.5,4.0])|     18.35| 23.43063341870668|
|(6,[0,1],[2.72,2.0])|     13.28| 17.08952944866254|
| (6,[0,1],[3.0,2.0])|      14.0| 18.01124667008746|
|[1.0,2.0,1.0,1.0,...|      5.75| 12.61966485036387|
|[1.1,2.0,1.0,1.0,...|      12.9|13.318891212190229|
|[1.17,2.0,0.0,1.0...|     32.83|14.271117703161945|
|[1.32,2.0,0.0,0.0...|      9.68|12.357596128241656|
|[1.36,3.0,1.0,0.0...|     18.64| 13.05805251860827|
|[1.48,2.0,0.0,0.0...|      8.52|10.642216191395434|
|[1.5,2.0,0.0,0.0,...|     19.08| 10.708053135

In [18]:
### Performance Metrics
pred_results.r2, pred_results.meanAbsoluteError, pred_results.meanSquaredError

(0.4104896090897727, 4.6501784594291475, 39.36079658675949)

### TO DO

How I ca  save this particular file in pickle format or in a temporary model pickle file. .save but check in documentation